In [1]:
import os

os.chdir("/data/notebook_files/Pengi")

In [2]:
from Pengi.wrapper import PengiWrapper as Pengi

pengi = Pengi(config="base")

generated_responses = pengi.generate(audio_paths=["/data/notebook_files/violin_0.wav"],
                                   text_prompts=["generate audio caption "],
                                   add_texts=[","],
                                   max_len=30,
                                   beam_size=6,
                                   temperature=1.0,
                                   stop_token=' <|endoftext|>'
                                   )

/opt/python/envs/minimal/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/python/envs/minimal/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: 100%|██████████| 4.01k/4.01k [00:00<00:00, 11.5MB/s]
Downloading: 100%|██████████| 571M/571M [00:11<00:00, 53.1MB/s]
Downloading: 100%|██████████| 665/665 [00:00<00:00, 3.19MB/s]
Downloading: 100%|██████████| 523M/523M [00:10<00:00, 54.2MB/s]
Downloading: 100%|██████████| 905/905 [00:00<00:00, 4.31MB/s]
Downloading: 100%|██████████| 939k/939k [00:00<00:00, 41.8MB/s]
Downloading: 100%|██████████| 512k/

In [8]:
pengi_output_prompts = generated_responses[0][0]
pengi_output_prompts

[' a violin is playing a note f. ',
 ' a violin is playing a melody. ',
 ' a violin is playing a violin. ',
 ' a violin is playing a note f#. ',
 ' a violin is playing a piece of a violin. ',
 ' a violin is playing a piece of a violin ']

In [5]:
from sentence_transformers import SentenceTransformer

In [10]:
goal_prompt = "A robotic sounding violin"
model = SentenceTransformer('all-MiniLM-L6-v2')

goal_encoded = model.encode(goal_prompt)

for prompt in pengi_output_prompts:
    prompt_encoded = model.encode(prompt)
    # new_encoded_prompt = prompt_encoded + (goal_encoded - prompt_encoded)


(384,)
(384,)
(384,)
(384,)
(384,)
(384,)


In [3]:
"""make variations of input image"""
import os, sys
os.chdir("/data/notebook_files/Make-An-Audio-main")

print(os.path.abspath("."))
import argparse, os, sys, glob
import PIL
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange, repeat
from torchvision.utils import make_grid
from torch import autocast
import librosa
# from contextlib import nullcontext
import time
from pytorch_lightning import seed_everything
import math
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from vocoder.bigvgan.models import VocoderBigVGAN
# from ldm.data.extract_mel_spectrogram import TRANSFORMS_22050,TRANSFORMS_16000
from preprocess.NAT_mel import MelNet
import soundfile

batch_max_length = 624
SAMPLE_RATE= 16000

def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(config, ckpt, verbose=True):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

def load_audio(path,transform,sr=16000,batch_max_length=624):# load wav and return mel
    wav,_ = librosa.load(path,sr=sr)

    audio = transform(wav) # (1,melbins,T)
    if audio.shape[2] <= batch_max_length:
        n_repeat = math.ceil((batch_max_length + 1) / audio.shape[1])
        audio = audio.repeat(1,1, n_repeat)

    audio = audio[..., :batch_max_length].unsqueeze(0) # shape [1,1,80,batch_max_length]
    return audio

def load_img(path):# load mel
    audio = np.load(path)
    if audio.shape[1] <= batch_max_length:
        n_repeat = math.ceil((batch_max_length + 1) / audio.shape[1])
        audio = np.tile(audio, reps=(1, n_repeat))

    audio = audio[:, :batch_max_length]
    audio = torch.FloatTensor(audio)[None, None, :, :] # [1,1,80,batch_max_length]
    return audio


class Opt:
    def __init__(self):
        self.config = "/data/notebook_files/Make-An-Audio-main/configs/text_to_audio/txt2audio_args.yaml"
        self.ckpt = "/data/notebook_files/Make-An-Audio-main/useful_ckpts/maa1_full.ckpt"
        self.vocoder_ckpt = "/data/notebook_files/Make-An-Audio-main/useful_ckpts/bigvnat"
        self.outdir = "/data/notebook_files/Make-An-Audio-main/outputs/audio2audio-samples"
        self.from_file = None
        
        self.strength = 0.3
        self.seed = 42
        self.scale = 3.0
        self.n_samples = 2
        self.n_iter = 1

        self.ddim_steps = 100
        self.ddim_eta = 0.0

        self.prompt = "a bird chirping"
        self.init_audio = "/data/notebook_files/violin_0.wav"

def main():
    opt = Opt()
    seed_everything(opt.seed)

    config = OmegaConf.load(f"{opt.config}")
    model = load_model_from_config(config, f"{opt.ckpt}")

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)

    hparams = {
        'audio_sample_rate': SAMPLE_RATE,
        'audio_num_mel_bins':80,
        'fft_size': 1024,
        'win_size': 1024,
        'hop_size': 256,
        'fmin': 0,
        'fmax': 8000,
        'batch_max_length': 1248, 
        'mode': 'pad', # pad,none,
    }
    melnet = MelNet(hparams)
    sampler = DDIMSampler(model)
    vocoder = VocoderBigVGAN(opt.vocoder_ckpt,device)

    os.makedirs(opt.outdir, exist_ok=True)
    outpath = opt.outdir

    batch_size = opt.n_samples # 一个prompt产生n_samples个结果
    if not opt.from_file: # load prompts from this file
        prompt = opt.prompt
        assert prompt is not None
        data = [batch_size * [prompt]]
    else:
        print(f"reading prompts from {opt.from_file}")
        with open(opt.from_file, "r") as f:
            data = f.read().splitlines()
            data = list(chunk(data, batch_size))


    sample_path = os.path.join(outpath, "samples")
    os.makedirs(sample_path, exist_ok=True)
    base_count = len(os.listdir(sample_path))

    assert os.path.isfile(opt.init_audio)
    init_image = load_audio(opt.init_audio,transform=melnet).to(device)
    init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
    init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
    sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)

    assert 0. <= opt.strength <= 1., 'can only work with strength in [0.0, 1.0]'
    t_enc = int(opt.strength * opt.ddim_steps)
    print(f"target t_enc is {t_enc} steps")

    with torch.no_grad():
        with model.ema_scope():
            tic = time.time()
            all_samples = list()
            for n in trange(opt.n_iter, desc="Sampling"):
                for prompts in tqdm(data, desc="data"):
                    uc = None
                    if opt.scale != 1.0: # default=5.0
                        uc = model.get_learned_conditioning(batch_size * [""])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    c = model.get_learned_conditioning(prompts)
                    z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device)) # [B, channel, c, h]
                    # decode it
                    samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                                unconditional_conditioning=uc,)

                    x_samples = model.decode_first_stage(samples)
                    print(x_samples.shape)
                    for x_sample in x_samples:
                        spec = x_sample[0].cpu().numpy()
                        spec_ori = init_image[0][0].cpu().numpy()
                        print(x_sample.shape,spec.shape,init_image.shape)
                        wav = vocoder.vocode(spec)
                        wav_ori = vocoder.vocode(spec_ori)
                        soundfile.write(os.path.join(outpath, f'{prompt.replace(" ", "-")}.wav'), wav, SAMPLE_RATE, 'FLOAT')
                        soundfile.write(os.path.join(outpath, f'{prompt.replace(" ", "-")}_ori.wav'), wav_ori, SAMPLE_RATE, 'FLOAT')
                        base_count += 1
                    all_samples.append(x_samples)


    print(f"Your samples are ready and waiting for you here: \n{outpath} \n"
            f" \nEnjoy.")


/data/notebook_files/Make-An-Audio-main
Loading model from /data/notebook_files/Make-An-Audio-main/useful_ckpts/maa1_full.ckpt
Global Step: 3722264
LatentDiffusion_audio: Running in eps-prediction mode
DiffusionWrapper has 160.22 M params.
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 78, 78) = 24336 dimensions.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
TextEncoder comes with 111.32

Global seed set to 42
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/python/envs/minimal/lib/python3.8/site-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecat

In [4]:
opt = Opt()
seed_everything(opt.seed)

config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

hparams = {
    'audio_sample_rate': SAMPLE_RATE,
    'audio_num_mel_bins':80,
    'fft_size': 1024,
    'win_size': 1024,
    'hop_size': 256,
    'fmin': 0,
    'fmax': 8000,
    'batch_max_length': 1248, 
    'mode': 'pad', # pad,none,
}
melnet = MelNet(hparams)
sampler = DDIMSampler(model)
vocoder = VocoderBigVGAN(opt.vocoder_ckpt,device)

os.makedirs(opt.outdir, exist_ok=True)
outpath = opt.outdir

batch_size = opt.n_samples # 一个prompt产生n_samples个结果
if not opt.from_file: # load prompts from this file
    prompt = opt.prompt
    assert prompt is not None
    data = [batch_size * [prompt]]
else:
    print(f"reading prompts from {opt.from_file}")
    with open(opt.from_file, "r") as f:
        data = f.read().splitlines()
        data = list(chunk(data, batch_size))


sample_path = os.path.join(outpath, "samples")
os.makedirs(sample_path, exist_ok=True)
base_count = len(os.listdir(sample_path))

assert os.path.isfile(opt.init_audio)
init_image = load_audio(opt.init_audio,transform=melnet).to(device)
init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)

assert 0. <= opt.strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(opt.strength * opt.ddim_steps)
print(f"target t_enc is {t_enc} steps")

with torch.no_grad():
    with model.ema_scope():
        tic = time.time()
        all_samples = list()
        for n in trange(opt.n_iter, desc="Sampling"):
            for prompts in tqdm(data, desc="data"):
                uc = None
                
                if opt.scale != 1.0: # default=5.0
                    uc = model.get_learned_conditioning(batch_size * [""])
                if isinstance(prompts, tuple):
                    prompts = list(prompts)
                c = model.get_learned_conditioning(prompts)
                # c = model.get_learned_conditioning(prompts) + model.get_learned_conditioning(goal) - model.get_learned_conditioning(prompt)
                z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device)) # [B, channel, c, h]
                # decode it
                samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                            unconditional_conditioning=uc,)

                x_samples = model.decode_first_stage(samples)
                print(x_samples.shape)
                for x_sample in x_samples:
                    spec = x_sample[0].cpu().numpy()
                    spec_ori = init_image[0][0].cpu().numpy()
                    print(x_sample.shape,spec.shape,init_image.shape)
                    wav = vocoder.vocode(spec)
                    wav_ori = vocoder.vocode(spec_ori)
                    soundfile.write(os.path.join(outpath, f'{prompt.replace(" ", "-")}.wav'), wav, SAMPLE_RATE, 'FLOAT')
                    soundfile.write(os.path.join(outpath, f'{prompt.replace(" ", "-")}_ori.wav'), wav_ori, SAMPLE_RATE, 'FLOAT')
                    base_count += 1
                all_samples.append(x_samples)


print(f"Your samples are ready and waiting for you here: \n{outpath} \n"
        f" \nEnjoy.")

Loading model from /data/notebook_files/Make-An-Audio-main/useful_ckpts/maa1_full.ckpt
Global Step: 3722264
LatentDiffusion_audio: Running in eps-prediction mode
DiffusionWrapper has 160.22 M params.
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 78, 78) = 24336 dimensions.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
TextEncoder comes with 111.32 M params.
target t_enc is 30 steps
torc

Global seed set to 42
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Sampling: 100%|██████████| 1/1 [00:06<00:00,  6.64s/it]


In [15]:
print(model.cond_stage_model)

FrozenCLAPEmbedder(
  (caption_encoder): TextEncoder(
    (base): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bia

In [18]:
c

tensor([[[ 0.5392, -1.1894,  0.9112,  ..., -1.7570,  0.4956, -0.9635],
         [-0.4502, -1.1678,  1.1993,  ...,  0.6420,  2.6234, -0.5755],
         [-1.1369, -1.6566,  0.3870,  ...,  0.2241,  1.6512,  1.1209],
         ...,
         [ 1.9683,  0.5220, -0.0055,  ..., -1.1538, -0.7219,  0.2007],
         [ 1.9511,  0.8519, -0.1219,  ..., -1.0268, -0.8133,  0.2305],
         [ 2.0007,  0.7828, -0.3059,  ..., -1.1670, -0.9217,  0.0657]],

        [[ 0.5392, -1.1894,  0.9112,  ..., -1.7570,  0.4956, -0.9635],
         [-0.4502, -1.1678,  1.1993,  ...,  0.6420,  2.6234, -0.5755],
         [-1.1369, -1.6566,  0.3870,  ...,  0.2241,  1.6512,  1.1209],
         ...,
         [ 1.9683,  0.5220, -0.0055,  ..., -1.1538, -0.7219,  0.2007],
         [ 1.9511,  0.8519, -0.1219,  ..., -1.0268, -0.8133,  0.2305],
         [ 2.0007,  0.7828, -0.3059,  ..., -1.1670, -0.9217,  0.0657]]],
       device='cuda:0')

In [17]:
print(model.cond_stage_model.caption_encoder)

TextEncoder(
  (base): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin